# $\left\langle L_{B,\tau, Z}\right\rangle$ Calculation

Here we want to provide the code to calculate $\left\langle L_{B,\tau, Z}\right\rangle$ for a given set of isochrones. Since this can be an expensive task, we're going to use Cython to speed up some parts of the code. Much of this notebook is identical to the notebook which calculates $f_{\rm det}$.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
%load_ext Cython

In [2]:
## a function for evaluating the IMF
def xi_IMF(ms,mpows,mcuts):
    res = np.zeros(ms.shape)
    (normx,normy) = (1.0,1.0)
    cuti = 0
    nms = len(ms)
    for i in range(nms):
        res[i] = normy*((ms[i]/normx)**mpows[cuti])
        if (i!=nms-1) and (ms[i+1]>mcuts[cuti+1]):
            normx = ms[i]
            normy = res[i]
            cuti +=1
    return res

## a function for calculating the IMF normalization
def norm_IMF(mpows,mcuts):
    res = 0
    (normx,normy) = (1.0,1.0)
    nmcs = len(mcuts)
    for i in range(nmcs-1):
        res += normy*(mcuts[i+1]**(mpows[i]+1) - mcuts[i]**(mpows[i]+1))/((normx**mpows[i])*(mpows[i]+1))
        normy = normy*((mcuts[i+1]/normx)**mpows[i])
        normx = mcuts[i+1]
    return res

In [3]:
# First specify the properties of the populations you'd like to calculate f_detect for
# by giving lists of metallicities and ages. These should be linearly spaced (in the log 
# space in which they are defined)
metlist = np.array([0.5,0.25,0.0,-0.25,-0.5,-0.75,-1.0,-1.25,-1.5,-1.75,-2.0,-2.25,-2.5,-2.75,-3.,-3.25])
agelist = np.array([8.95,9.,9.05,9.1,9.15,9.2,9.25,9.3,9.35,9.4,9.45,9.5,9.55,9.6,9.65,9.7,9.75,9.8,9.85,9.9,9.95,10.,10.05,10.1])
nmet = len(metlist) # number of metallicities
nage = len(agelist) # number of ages
dage = agelist[1] - agelist[0] # logarithmic difference in age bins, for selection below

## specify directory where isochrones are stored, this is set up for the 
## MIST Roman Isocrones, changing that would involve changing the below
iso_dir = "../../mist_isos/"
## name od output file
output_name = "../data/lum_mist" # it will have a .npy at the end

## define the IMF normalization
## specify the power-law indices for the IMF that you would like to use
## here we define a Kroupa IMF
mpows = [-1.3,-2.3]
mcuts = [0.08,0.5,120]
assert(len(mpows)+1==len(mcuts))
## specify the number of mass samples you would like for integrals of the IMF
nmass_samp = 100000
# mass range in solar masses
(mMin,mMax) = (mcuts[0],mcuts[-1])
# masses spaced linearly throughout imf space
mlin = np.linspace(mMin,mMax,nmass_samp)
# evaluate IMF
xi = xi_IMF(mlin,mpows,mcuts)
# the IMF normalization over the mass range of interest
xim_norm = norm_IMF(mpows,mcuts)
xi = xi/xim_norm

# specify the number of filters. The default is 7 (R,Z,Y,J,W,H,F). Changing this 
# number will require some hard code changes below
nfilts = 7
# conversion from Vega mags to AB mags for each band
AB_Vega = np.array([0.147,0.485,0.647,0.950,1.012,1.281,1.546]) #This is m_AB - m_Vega

The following cell is where we define the function needed to compute the average luminosity, which is effectively the only cell which is different from the $f_{\rm det}$ calculation. We provide an implementation first in Python and then in Cython.

In [6]:
def get_Lavg_python(B_mu, ms, xiofm):
    fsum = 0
    
    for i in range(len(B_mu)-1):
        fsum += (10**(B_mu[i]/-2.5)*xiofm[i] + 10**(B_mu[i+1]/-2.5)*xiofm[i+1])*(ms[i+1] - ms[i])/2.
    return fsum

In [7]:
%%cython
# cython: boundscheck=False, wraparound=False, nonecheck=False, cdivision=True
import numpy as np
cimport numpy as np


cimport cython

cpdef double get_Lavg(const double[::1] B_mu, const double[::1] ms, 
                      const double[::1] xiofm, const int N):
    cdef:
        double fsum = 0
        double aB,bB,aX,bX,mcross,xicross
        int i
    
    for i in range(N-1):
        fsum += (10**(B_mu[i]/-2.5)*xiofm[i] + 10**(B_mu[i+1]/-2.5)*xiofm[i+1])*(ms[i+1] - ms[i])/2.
    return fsum

In [5]:
%%time
# shape of the total output array
totdat = np.zeros((nmet,nage,nfilts))
for (l,met) in enumerate(metlist):
    print(met)
    if met>-0.1:
        (age,imass,T,L) = np.loadtxt("%smist_fehp%1.2f.iso.cmd"%(iso_dir,abs(met)),usecols=[1,2,4,6]).T
        (R,Z,Y,J,W,H,F) = np.loadtxt("%smist_fehp%1.2f.iso.cmd"%(iso_dir,abs(met)),usecols=[9,10,11,12,13,14,15]).T
    else:
        (age,imass,T,L) = np.loadtxt("%smist_fehm%1.2f.iso.cmd"%(iso_dir,abs(met)),usecols=[1,2,4,6]).T
        (R,Z,Y,J,W,H,F) = np.loadtxt("%smist_fehm%1.2f.iso.cmd"%(iso_dir,abs(met)),usecols=[9,10,11,12,13,14,15]).T

    (Rage,Zage,Yage,Jage,Wage,Hage,Fage) = (np.zeros(nage),np.zeros(nage),np.zeros(nage),np.zeros(nage),np.zeros(nage),np.zeros(nage),np.zeros(nage))
    for (k,a) in enumerate(agelist):
        sids = np.intersect1d(np.where(age>a-dage/2),np.where(age<a+dage/2))

        # interpolate what the right magnitudes are
        R_out = R[sids] + AB_Vega[0]
        Z_out = Z[sids] + AB_Vega[1]
        Y_out = Y[sids] + AB_Vega[2]
        J_out = J[sids] + AB_Vega[3]
        W_out = W[sids] + AB_Vega[4]
        H_out = H[sids] + AB_Vega[5]
        F_out = F[sids] + AB_Vega[6]
        mass_sel = imass[sids]
        xim = np.interp(imass[sids],mlin,xi)
        Rage[k] = get_Lavg(R_out,mass_sel,xim,len(xim))
        Zage[k] = get_Lavg(Z_out,mass_sel,xim,len(xim))
        Yage[k] = get_Lavg(Y_out,mass_sel,xim,len(xim))
        Jage[k] = get_Lavg(J_out,mass_sel,xim,len(xim))
        Wage[k] = get_Lavg(W_out,mass_sel,xim,len(xim))
        Hage[k] = get_Lavg(H_out,mass_sel,xim,len(xim))
        Fage[k] = get_Lavg(F_out,mass_sel,xim,len(xim))

    totdat[l] = np.array([Rage,Zage,Yage,Jage,Wage,Hage,Fage]).T
np.save(output_name,totdat)

0.5
0.25
0.0
-0.25
-0.5
-0.75
-1.0
-1.25
-1.5
-1.75
-2.0
-2.25
-2.5
-2.75
-3.0
-3.25
CPU times: user 35.7 s, sys: 1.38 s, total: 37 s
Wall time: 37.3 s
